In [1]:
import numpy as np
import pandas as pd
import sys
import random
import progressbar
import torch
import pickle
from mytree import *
from utils import *
from treeUtil import *
import tqdm
import argparse
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
import functools
from sklearn.linear_model import LogisticRegression
import gensim
from gensim.models import KeyedVectors
import string
from stanfordcorenlp import StanfordCoreNLP
import nltk.tree
from ast import literal_eval
import pptree
import logging

In [2]:
from nltk.tokenize import word_tokenize
import re
import os
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from logger import Logger

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from spacy.pipeline import merge_entities
nlp = en_core_web_sm.load()
nlp.add_pipe(merge_entities)

In [4]:
import copy
from sklearn.utils import shuffle
from sklearn import metrics

In [18]:
# False if in-domain; True if general
proanti = True
w2vec = False
ner = True
blackout = False
balanced = True
undersample = False
non_trainable = True
economic = True

In [19]:
dic = {0:'ProAnti', 1:'General', 2:'NER', 3:'Blackout', 4:'Balance', 5:'Undersample', 6:'Fixed'}
bool_list = [proanti, w2vec, ner, blackout, balanced, undersample, non_trainable]
namecode = 'economic_w2v'
for index, bb in enumerate(bool_list):
    namecode += '_'
    if bb:
        namecode += '1'
    else:
        namecode += '0'
    namecode += dic[index]
namecode

'economic_w2v_1ProAnti_0General_1NER_0Blackout_1Balance_0Undersample_1Fixed'

In [7]:
CUDA=False
def Var(v):
    if CUDA: return Variable(v.cuda())
    else: return Variable(v)
widgets = [progressbar.Percentage(), ' ', progressbar.Bar(), ' ', progressbar.ETA()]

In [8]:
class RecursiveNN(nn.Module):
    def __init__(self, word_embeddings, vocab, embedSize=300, numClasses=2, beta = 0.3, use_weight = True, non_trainable = non_trainable):
        super(RecursiveNN, self).__init__()
#             if (w2vec):
#                 self.embedding = nn.Embedding(len(vocab), embedSize)
#                 self.embedding.load_state_dict({'weight': w2vec_weights})
        self.embedding = nn.Embedding.from_pretrained(word_embeddings)
        self.embedding.weight.requires_grad = True
        if non_trainable:
            self.embedding.weight.requires_grad = False
        else:
            self.embedding = nn.Embedding(len(vocab), embedSize)
        self.embedding = nn.Embedding(len(vocab), embedSize)
        self.W = nn.Linear(2*embedSize, embedSize, bias=True)
        self.nonLinear = torch.tanh
        self.projection = nn.Linear(embedSize, numClasses, bias=True)
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
        self.V = vocab
        self.beta = beta
        self.use_weight = use_weight
        self.total_rep = None #
        self.count_rep = 0 #
        self.numClasses = numClasses

    def traverse(self, node):
        if node.isLeaf:
            if node.getLeafWord() in self.V:  # check if right word is in vocabulary
                word = node.getLeafWord()
            else:  # otherwise use the unknown token
                word = 'UNK'
            # print(self.V[word],len(self.V),word,(torch.LongTensor([int(self.V[word])])))
            currentNode = (self.embedding(Var(torch.LongTensor([int(self.V[word])]))))
        else: currentNode = self.nonLinear(self.W(torch.cat((self.traverse(node.left),self.traverse(node.right)),1)))
        currentNode = currentNode/(torch.norm(currentNode))

        assert node.label!=None
        self.nodeProbList.append(self.projection(currentNode))
#         print(node.label)
#         print(torch.LongTensor([node.label]))
        self.labelList.append(torch.LongTensor([node.label]))
        loss_weight = 1-self.beta if node.annotated else self.beta
        self.loss += (loss_weight*F.cross_entropy(input=torch.cat([self.projection(currentNode)]),target=Var(torch.cat([torch.LongTensor([node.label])]))))

        #
        if not node.isRoot():
            if self.total_rep is None:
                self.total_rep = currentNode.data.clone()
            else:
                self.total_rep += currentNode.data.clone()
            self.count_rep += 1
        #
        return currentNode   
    
    
    
    def traverse_test(self, node):
        if node.isLeaf:
            if node.getLeafWord() in self.V:  # check if right word is in vocabulary
                word = node.getLeafWord()
            else:  # otherwise use the unknown token
                word = 'UNK'
            # print(self.V[word],len(self.V),word,(torch.LongTensor([int(self.V[word])])))
            currentNode = (self.embedding(Var(torch.LongTensor([int(self.V[word])]))))
        else: currentNode = self.nonLinear(self.W(torch.cat((self.traverse_test(node.left),self.traverse_test(node.right)),1)))
        currentNode = currentNode/(torch.norm(currentNode))

#         assert node.label!=None
        self.nodeProbList.append(self.projection(currentNode))
        loss_weight = 1-self.beta if node.annotated else self.beta

        if not node.isRoot():
            if self.total_rep is None:
                self.total_rep = currentNode.data.clone()
            else:
                self.total_rep += currentNode.data.clone()
            self.count_rep += 1
        #
        return currentNode
        

    def forward(self, x):
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
#         self.traverse(x)
        self.traverse_test(x)
#         self.labelList = Var(torch.cat(self.labelList))
        return torch.cat(self.nodeProbList)

    def getLoss(self, tree):
        nodes = self.forward(tree)
        predictions = nodes.max(dim=1)[1]
        loss = self.loss
        return predictions,loss
    
    
    def predict(self, trees):
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
        preds = []
        for j, tree in enumerate(trees):
            nodes = self.forward(tree.root)
            predictions = nodes.max(dim=1)[1]
#                     print((predictions.cpu().data).numpy(),(predictions.cpu().data).numpy().shape)
#                     print((self.labelList.cpu().data).numpy(), (self.labelList.cpu().data).numpy().shape)
            preds.append(predictions)
            pbar.update(j)
        pbar.finish()
        return preds
    

    def getRep(self, tree):
        self.count_rep = 0
        self.total_rep = None
        self.nodeProbList = []
#         self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))

        root_rep = self.traverse(tree)

        return (torch.cat((root_rep,self.total_rep/self.count_rep),1)).data.numpy().T.flatten()

    
#     def evaluate(self, trees):
#         pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
#         n = nAll = correctRoot = correctAll = 0.0
#         tp = [1e-2]*self.numClasses
#         fp = [1e-2]*self.numClasses
#         fn = [1e-2]*self.numClasses
#         f1 = [0.]*self.numClasses
#         for j, tree in enumerate(trees):
#             predictions,_ = self.getLoss(tree.root)
# #                     print((predictions.cpu().data).numpy(),(predictions.cpu().data).numpy().shape)
# #                     print((self.labelList.cpu().data).numpy(), (self.labelList.cpu().data).numpy().shape)
#             correct = ((predictions.cpu().data).numpy()==(self.labelList.cpu().data).numpy())
# #                     print(correct)
#             correctAll += correct.sum()
#             nAll += np.shape(correct.squeeze())[0] if np.size(correct)!=1 else 1 
#             correctRoot += correct.squeeze()[-1] if np.size(correct)!=1 else correct[-1]
# #                     print(correct.squeeze()[-1] if np.size(correct)!=1 else correct[-1])
# #                     print('actual: {}'.format(tree.root.label))
#             for i in range(self.numClasses):
#                 size = np.size((predictions.cpu().data).numpy())
#                 if size!=1:
#                     pred = (predictions.cpu().data).numpy().squeeze()[-1]
#                     actual = (self.labelList.cpu().data).numpy().squeeze()[-1]
#                 else:
#                     pred = (predictions.cpu().data).numpy()[-1]
#                     actual = (self.labelList.cpu().data).numpy()[-1]
#                 if pred==i and actual==i:
#                     tp[i]+=1
#                 elif pred==i and actual!=i:
#                     fn[i]+=1
#                 elif pred==i and actual!=i:
#                     fp[i]+=1
#             n += 1
#             pbar.update(j)
# #             print(tp,fp,fn)
#         for i in range(self.numClasses):
#             p =(1.0*tp[i]/(tp[i]+fp[i]))
#             r =(1.0*tp[i]/(tp[i]+fn[i]))
#             f1[i] = (2*p*r)/(p+r)
#         pbar.finish()
#         return correctRoot / n, correctAll/nAll, f1

    def eval_sent_lvl(self,trees,clf):
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
        n = nAll = correctRoot = correctAll = 0.0
        X_predict = []
        Y_gold = []
        for j, tree in enumerate(trees):
            tree_rep = model.getRep(tree.root)
            X_predict.append(tree_rep)
            Y_gold.append(tree.root.label)
        acc = clf.score(np.array(X_predict),np.array(Y_gold))
        return acc

## Process data

In [9]:
def replace_entity(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            sent = sent.replace(ent.text,ent.label_)
    return sent

def blackout_entity(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            sent = sent.replace(ent.text,"")
    return sent

def read_into_string(filename):
    text_file = open(filename, 'r')
    lines = text_file.read().split('\n')
    if ner:
        lines = [replace_entity(sent) for sent in lines]
    if blackout:
        lines = [blackout_entity(sent) for sent in lines]
    text_file.close()
    return lines

def process(file):
    sents = read_into_string(file)
    sents = [s for s in sents if len(s)>14 and len(s)<500]
    return sents

In [13]:
def make_tree(text):
#     print(text)
    output = nlp.annotate(text, properties={
        'annotators': 'tokenize,ssplit,pos,depparse,parse',
        'outputFormat': 'json'
    })
    output = literal_eval(output)
    try:
        tree = str(output['sentences'][0]['parse'])
    except:
        print(output,text)
        return
    # print (tree)
    parse_string = ' '.join(str(tree).split())
    # print(parse_string)
    # print ("\n\n")
    tree = nltk.tree.Tree.fromstring(parse_string)
    tree.chomsky_normal_form()
    tree.collapse_unary(collapseRoot=True,collapsePOS=True)
    nt = convertNLTK_tree(tree)
    return nt

def printLabelTree(tree):
    def inorder(node,nnode):
        if node.isLeaf:
            newnode = pptree.Node('H',nnode)
            wnode = pptree.Node(node.word,newnode)
        elif nnode is not None:
            newnode = pptree.Node('H',nnode)
            inorder(node.left,newnode)
            inorder(node.right,newnode)
        elif node.isRoot():
            newnode = pptree.Node('H')
            inorder(node.left,newnode)
            inorder(node.right,newnode)
            return newnode
        return None
    pptree.print_tree(inorder(tree.root,None))

def create_trees_using_df(df):
    tree = []
    for tokens in list(df['tokens']):
        if len(tokens)==0:
            continue
        line = ' '.join(tokens)
        line += '\n'
        tree.append(make_tree(line))
    return tree

def printlabel(root,l):
    if root:
        l.append(root.label)
#         print(root.label)
        if root.left:
            l+=printlabel(root.left,[])
#             print(printlabel(root.left))
        if root.right:
            l+=printlabel(root.right,[])
#             print(printlabel(root.right))
    return l

In [10]:
# path of text file to process
file_path = 'sample.txt'
sentences = process(file_path)
# length of a sentence should be between 14 and 500, else parse trees cannot be formed - stanford parser throws error
sentences = [sent for sent in sentences if len(sent)>14 and len(sent)<500]
len(sentences)

4

Run this command in the terminal, from stanford-corenlp-full-2018-10-05 directory

- java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "tokenize,ssplit,pos,lemma,parse,sentiment" -port 9000 -timeout 90000

In [11]:
nlp = StanfordCoreNLP('http://localhost', port=9000,timeout=90000)

In [22]:
df = pd.DataFrame({'statement':sentences})
df['tokens'] = [word_tokenize(re.sub(r'[^\w\s]|[\d]+',' ',sent)) for sent in df['statement']]
df.head()

,statement,tokens
0,This makes it not only one of the most accurat...,"[This, makes, it, not, only, one, of, the, mos..."
1,Acknowledging slip-ups in the Aadhaar enrolmen...,"[Acknowledging, slip, ups, in, the, Aadhaar, e..."
2,"Briefing reporters here , UIDAI chairman PERSO...","[Briefing, reporters, here, UIDAI, chairman, P..."
3,Modi said the banks should redouble efforts in...,"[Modi, said, the, banks, should, redouble, eff..."


In [23]:
trees = create_trees_using_df(df)

Store the model in a subfolder 'models'

In [25]:
model = pickle.load(open("./models/"+namecode+'.pkl','rb'))
preds = model.predict(trees)
preds = [p.numpy()[-1] for p in preds]
labels = []
i=0
for p in preds:
    if p==0:
        labels.append('PRO')
    else:
        labels.append('ANTI')
        i+=1

df_preds = pd.DataFrame({"Statement": df['statement'], "Label":labels})
df_preds.to_csv('predictions.csv')
df_preds.head()

100% |##########################################################| Time: 0:00:00


,Statement,Label
0,This makes it not only one of the most accurat...,PRO
1,Acknowledging slip-ups in the Aadhaar enrolmen...,ANTI
2,"Briefing reporters here , UIDAI chairman PERSO...",PRO
3,Modi said the banks should redouble efforts in...,ANTI
